In [14]:
# !pip install openpyxl

In [43]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [44]:
df = pd.read_csv("/data/Augmented_data.csv")
df.shape

(234501, 18)

In [45]:
df[df["Exited"] == 1].shape[0] /df.shape[0]

0.14290770615050682

In [46]:
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15572489,HEALEY,1720.007287,Spain,Female,73,29,393987.727351,1,1,1,1.0,0,0,3,DIAMOND,2531.0,1-6-2022
1,15800464,PARTIDA,1518.782461,Spain,Male,53,15,274502.817582,2,1,0,1.0,1,0,4,SILVER,1725.0,1-2-2022
2,15580134,Crawford,502.950000,Spain,Male,27,2,184535.891500,1,1,1,1.0,0,0,3,DIAMOND,711.0,1-1-2020
3,15074588,PINKERTON,1468.954354,Spain,Male,49,15,0.000000,1,1,0,1.0,0,0,5,SILVER,1512.0,1-1-2022
4,15982065,ABERNATHY,1683.914742,France,Male,49,30,0.000000,2,1,0,1.0,1,0,3,DIAMOND,1296.0,1-6-2022


In [47]:
cities = pd.read_excel("US_cities.xlsx")
# companies = pd.read_csv("us_companies.csv")

In [48]:
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)

In [49]:
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())

100%|██████████| 234501/234501 [00:00<00:00, 995919.88it/s] 


In [50]:
state_mapper = {}
def get_state(x):
    if x not in state_mapper:
        state = np.random.choice(list(state_city.keys()))
        state_mapper[x] = state
        return state
    else:
        return state_mapper[x]
    
df["Geography"] = df["Geography"].progress_apply(get_state)

100%|██████████| 234501/234501 [00:00<00:00, 1222518.23it/s]


In [51]:
df["Geography"].unique()

array(['New York', 'Nebraska', 'Pennsylvania'], dtype=object)

In [53]:
df["City"] = df["Geography"].progress_apply(lambda x: np.random.choice(state_city[x]))

100%|██████████| 234501/234501 [00:04<00:00, 53738.60it/s]
